In [ ]:
%matplotlib inline
from importlib import reload
import json
import sys
sys.path.append("..")
from pathlib import Path
from tqdm import tqdm

import numpy as np
import scipy as sp
import pandas as pd
from matplotlib import pyplot as plt
from skimage import io
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import cv2

import pythd

In [ ]:
DATA_DIR = Path("..") / Path("data")
CHIP_SHAPE = (16, 16)
CHIP_STRIDE = (16, 16)

In [ ]:
reload(pythd)
reload(pythd.image)

DAMAGE_LEVELS = {
    "un-classified": 1,
    "no-damage": 2,
    "minor-damage": 3,
    "major-damage": 4,
    "destroyed": 5
}

def process_image(pre_img, post_img, chip_size, chip_stride):
    """
    Process and chip a single image
    """
    # Color transfer
    pre_img = cv2.cvtColor(pythd.image.normalize_image(pre_img), cv2.COLOR_RGB2Lab)
    post_img = cv2.cvtColor(pythd.image.normalize_image(post_img), cv2.COLOR_RGB2Lab)
    for i in range(pre_img.shape[2]):
        post_img[:,:,i] = ((post_img[:,:,i] - post_img[:,:,i].mean()) / post_img[:,:,i].std() * pre_img[:,:,i].std()) + pre_img[:,:,i].mean()
    pre_img = pythd.image.normalize_image(cv2.cvtColor(pre_img, cv2.COLOR_Lab2RGB))
    pre_img = (pre_img * 255.0).astype(np.uint8)
    post_img = pythd.image.normalize_image(cv2.cvtColor(post_img, cv2.COLOR_Lab2RGB))
    post_img = (post_img * 255.0).astype(np.uint8)
    
    dfs = []
    for k, img in {"pre": pre_img, "post": post_img}.items():
        # RGB
        df = pythd.image.ChippedImage.FromRGB(img, chip_size, chip_stride).df
        cols = [col for col in df.columns if "pixel" in col]
        df.loc[:, cols] = (df.loc[:, cols] - df.loc[:, cols].mean(axis=0)) / df.loc[:, cols].std(axis=0)
        df.columns = [k + "_" + col for col in df.columns]
        dfs.append(df)

        # Edges
        edges = cv2.Canny(img, threshold1=100, threshold2=200, apertureSize=3, L2gradient=True)
        edges = pythd.image.normalize_image(edges)
        edges_df = pythd.image.ChippedImage.FromGrayscale(edges, chip_size, chip_stride).df
        cols = [col for col in edges_df.columns if "pixel" in col]
        edges_df = edges_df.loc[:, cols]
        edges_df.columns = [k + "_edge " + col for col in edges_df.columns]
        dfs.append(edges_df)
    
    df = pd.concat(dfs, axis=1)
    df.drop(["post_x", "post_y"], axis=1, inplace=True)
    df.rename({"pre_x": "x", "pre_y": "y"}, axis=1, inplace=True)
    return df

def build_chipped_df(images, chip_size=(16,16), chip_stride=(16,16)):
    """
    Build a chipped dataframe from a list of image paths
    """
    dfs = []
    cw, ch = chip_size
    sx, sy = chip_stride
    
    for pre_path in tqdm(list(images)):
        #pre disaster image
        pre_img = io.imread(str(pre_path))
        h, w = pre_img.shape[:2]
        post_path = DATA_DIR / pre_path.name.replace("pre", "post")
        post_img = io.imread(str(post_path))
        df = process_image(pre_img, post_img, chip_size, chip_stride)
        df["image"] = pre_path.name
        df.astype({"x": "int32", "y": "int32"}, copy=False)
        df["building"] = 0
        df["damage"] = 1
        
        # labels
        label_path = DATA_DIR / post_path.name.replace("png", "json")
        if label_path.exists():
            label_mask = np.zeros((h, w), dtype=np.uint8)
            with open(label_path, "r") as f:
                labels = json.load(f)
                labels = labels["features"]["xy"]
                for o in [obj for obj in labels if (obj["properties"]["feature_type"] == "building")]:
                    coords = o["wkt"][10:-2].split(', ')
                    bbox= np.array([list(map(lambda x: int(float(x)), coord.split(' '))) for coord in coords])
                    damage = DAMAGE_LEVELS[o["properties"]["subtype"]]
                    cv2.fillPoly(label_mask, [bbox], color=damage)
            
            for row in df.index:
                x = int(df.loc[row, "x"])
                y = int(df.loc[row, "y"])
                chip = label_mask[y:(y+ch), x:(x+cw)]
                df.loc[row, "building"] = int((chip > 0).any())
                df.loc[row, "damage"] = chip.max()
        dfs.append(df)
    
    dfs = pd.concat(dfs, axis=0, ignore_index=True)
    return dfs

df = build_chipped_df(list(DATA_DIR.glob("*pre_disaster.png")), chip_size=CHIP_SHAPE, chip_stride=CHIP_STRIDE)

In [ ]:
features = [c for c in df.columns if "pixel" in c]
data = df.loc[:, features].values
print(df.shape)
print(df["image"].unique())
df.head()

In [ ]:
reload(pythd)
reload(pythd.filter)

filt1 = pythd.filter.ScikitLearnFilter(PCA, n_components=100)
filt1.fit(data)

In [ ]:
reload(pythd)
reload(pythd.filter)

V = np.var(filt1(data), axis=0)

filt2 = pythd.filter.ScikitLearnFilter(TSNE, n_components=2, 
                                       metric='correlation',
                                       init='pca', angle=0.7)
filt = pythd.filter.CompositeFilter(filt1, filt2)
f_x = filt(data)
print(f_x.shape)

In [ ]:
reload(pythd)
reload(pythd.thd)
reload(pythd.complex)
reload(pythd.cover)
reload(pythd.mapper)
reload(pythd.clustering)

clustering = pythd.clustering.HierarchicalClustering(metric='seuclidean', method='average')
cover = pythd.cover.IntervalCover.EvenlySpacedFromValues(f_x, 50, 0.3)
thd = pythd.thd.THD(data, filt, cover, group_threshold=200, contract_amount=0.05)
thd.run(verbose=True)
root = thd.get_results()

In [ ]:
import igraph
root.color_by_value(df["building"].values, normalize=True)
#root.color_density(normalize=True)
g = root.as_igraph_graph()
vs = {
    "margin": 40,
    "bbox": (650, 350),
    "vertex_label": g.vs["name"],
    "vertex_label_size": 10,
    "vertex_size": 35,
    "vertex_label_dist": 0,
    "vertex_label_angle": 0,
    "layout": g.layout_reingold_tilford(root=[0])
}
igraph.plot(g, **vs)

In [ ]:
def show_group_image(group, i=0, which='pre'):
    images = df["image"].iloc[list(group.rids)].unique()
    gdata = df.iloc[list(group.rids), :]
    img_name = images[i]
    gdata = gdata[gdata["image"] == img_name]

    img_path = DATA_DIR / img_name
    if which == 'pre':
        img_path = DATA_DIR / img_name.replace('post', 'pre')
    else:
        img_path = DATA_DIR / img_name.replace('pre', 'post')
    img = io.imread(str(img_path))
    mask = np.zeros(img.shape[:2], dtype=np.float32)
    cw, ch = CHIP_SHAPE

    for x, y in zip(gdata["x"].values, gdata["y"].values):
        x, y = (int(x), int(y))
        mask[y:(y+ch), x:(x+cw)] = 1

    overlay = pythd.image.overlay_mask(img, mask, image_alpha=0.6)

    pythd.image.plot_images((overlay,))

group = root.get_group_by_name("9.2.0")
images = df["image"].iloc[list(group.rids)].unique()
print(images)

In [ ]:
show_group_image(group, 0)

In [ ]:
show_group_image(group, 1)

In [ ]:
show_group_image(group, 2)

In [ ]:
show_group_image(group, 3)

In [ ]:
show_group_image(group, 4)

In [ ]:
show_group_image(group, 5)

In [ ]:
show_group_image(group, 6)

In [ ]:
root.color_by_value(df["damage"].values, normalize=True)
g = root.as_igraph_graph()
vs = {
    "margin": 40,
    "bbox": (650, 350),
    "vertex_label": g.vs["name"],
    "vertex_label_size": 10,
    "vertex_size": 35,
    "vertex_label_dist": 0,
    "vertex_label_angle": 0,
    "layout": g.layout_reingold_tilford(root=[0])
}
igraph.plot(g, **vs)

In [ ]:
group = root.get_group_by_name("2.19.0")
images = df["image"].iloc[list(group.rids)].unique()
print(images)

In [ ]:
show_group_image(group, 0, which='post')

In [ ]:
show_group_image(group, 1, which='post')

In [ ]:
show_group_image(group, 2, which='post')

In [ ]:
show_group_image(group, 3, which='post')

In [ ]:
show_group_image(group, 4, which='post')

In [ ]:
show_group_image(group, 5, which='post')

In [ ]:
show_group_image(group, 6, which='post')